In [1]:
"""
By:    David Malott
Org:   SpaceFactory.io
Date:  February 4 2018
Description:  This program generates a graphic color pulse using a simulated 
              asynchronous event stream 
"""


# Imports

import queue
import threading
import time
import random
import cv2
import numpy as np


# Global Variables

random.seed()                   # initialize random number generator

event = queue.Queue()           # create event queue

print_lock = threading.Lock()   # set print threading lock 

timeout = 30                    # set seconds to wait for signal until stopping pulse generator

                                # create color dictionary tied to signal key

color_dict = { 1 : "blue",
               2 : "cyan",
               3 : "green", 
               4 : "yellow",
               5 : "red",
               6 : "magenta",
               7 : "white", 
             }

                                #create color dictionary using format b, g, r
    
bgr_dict =  { 'blue'    : [255, 0, 0],
              'cyan'    : [255, 255, 0], 
              'green'   : [0, 255, 0],
              'yellow'  : [0, 255, 255],
              'red'     : [0, 0, 255],
              'magenta' : [255, 0, 255], 
              'white'   : [255, 255, 255],
              'black'   : [0, 0, 0]
             }
 
# Image Setup: 720 x 1280p is standard, apply scale factor if desired
    
img_h = 720              #image height
img_w = 1280             #image width
img_scale = 0.5          #image scale factor
img_blur = 51            #blurriness of the pulse -> change for effect

#scale image
img_h = int(img_h * img_scale)
img_w = int(img_w * img_scale)

# initialize blank image    
img = np.zeros((int(img_h), int(img_w), 3), np.uint8)

    
# Functions

def genPulse(color, speed):
    
    #Generate a color pulse using two arguments, color and speed
    
    #Bring in global variables
    
    global img_h
    global img_w
    global img
    global img_blur
    
    #Pulse Parameters: height, width, slope, and blur

    size = 40                #weight of the pulse in pixels
    x1 = int(img_w / 2)      #center of the pulse, set to center of the image
    x2 = 0                   #half-length of the pulse, initially set to 0
    x3 = -20                 #distance by which wake is shorter than pulse->change for effect
    slope = 3                #slope of the pulse -> calibrate for image proportion
    
    # get b, g, r value from color

    b, g, r = bgr_dict[color]

    for i in range (0, int(img_h / speed)):
        
        #set growth relative to slope and speed
        x2 = int(i / slope) * speed
        
        #set vertical motion relative to speed
        j = i * speed

        #create a pulse
        img[j : j + size, x1 - x2 : x1 + x2] = [b, g, r]
        
        #create a wake
        img[0 : j, x1 - x2 + x3 : x1 + x2 - x3] = [0 , 0, 0]

        img = cv2.blur(img, (img_blur, img_blur))
        
        cv2.imshow("Pulse",img)
        cv2.waitKey(1)
        


def initPulse():
    
    # Initialize a color pulse using signals in event queue 
    # If signal is a color, generatue a pulse with funciton genPulse
    # Speed of pulse is faster if more events are in queue
    
    pulse = 0                # count number of pulses
    timer = 0                # initialize timer for timeout
    
    time.sleep(1) # wait one second to receive first signal
    
    print ("Initialized pulse")
    
    # loop until timeout:
    
    while True: 

        if event.qsize() == 0:
            
            # wait one second to receive next signal and add to timer
        
            time.sleep(1) 
            timer = timer + 1 
            with print_lock:
                print ( "  waiting %d..." % timer )
        
        else:  
            
            # get an event from queue and generate color pulse if event is a color
              
            color = event.get()
            speed = event.qsize() + 1
            b, g, r = bgr_dict[color]   # get b, g, r value from color
            
            genPulse (color, speed)     # generate pulse
                          
            event.task_done()           # indicates that an enqueued task is complete
            pulse = pulse + 1           # count the pulse
            timer = 0                   # reset timer
                
            with print_lock:
                print( "  Pulse %d: %s speed: %d" % ( pulse, color, speed ) )
                    
        # break the loop if signal has not been received in time
                                        
        if timer == timeout:
            break
                               
    print ( "Timeout: stopped pulse generator" )
    

def fakeSignal():
    
    # This function generates a signal which could be blank, or a photon of random color
    
    for i in range(20):
    
        start = time.time()  # count the start time

        time.sleep( random.uniform(1.0, 5.0) ) # delay the signal by a random interval

        signal = random.randint(1, 8) # create a random signal in range      

        if signal in range (1, 8):      # if signal is good
            color = color_dict[signal]  # assign color by key
            event.put(color)            # put color to the queue
        
        end = time.time()  # count the end time

        with print_lock:
            print ( "  Signal %d: %d (delay: %f)" % (i + 1, signal, (end - start) ) )
            
    print ( "Signal ended" )
    
    print ( "Blocked queue" )
    event.join()


# Main 

# create threads

signalThread = threading.Thread(target = fakeSignal)

# classify threads as a daemon, so they will die when the main dies

signalThread.daemon = True

# start threads, must come after daemon definition

signalThread.start()
print ( "Started signal" )

# run pulse generator
initPulse()

# close threads, upon completion

signalThread.join()
print ( "Closed signal thread" )

#close windows and end program

cv2.destroyAllWindows()
print ( "Closed windows.  End of program" )


Started signal
Initialized pulse
  Signal 1: 2 (delay: 2.005987)  waiting 1...

  Signal 2: 1 (delay: 1.936434)
  Pulse 1: cyan speed: 1
  Signal 3: 5 (delay: 4.375295)
  Pulse 2: blue speed: 1
  Pulse 3: red speed: 1
  Signal 4: 6 (delay: 3.427410)
  waiting 1...
  Signal 5: 1 (delay: 1.961999)
  Pulse 4: magenta speed: 1
  Signal 6: 2 (delay: 2.596596)
  Pulse 5: blue speed: 1
  Pulse 6: cyan speed: 1
  waiting 1...
  Signal 7: 7 (delay: 4.838607)
  waiting 2...
  Signal 8: 3 (delay: 3.190585)
  Pulse 7: white speed: 1
  Signal 9: 5 (delay: 2.306393)
  Pulse 8: green speed: 1
  Pulse 9: red speed: 1
  waiting 1...
  Signal 10: 2 (delay: 3.882411)
  waiting 2...
  Pulse 10: cyan speed: 1
  Signal 11: 7 (delay: 4.367090)
  waiting 1...
  Pulse 11: white speed: 1
  Signal 12: 6 (delay: 3.276111)
  waiting 1...


KeyboardInterrupt: 

  Signal 13: 3 (delay: 4.395354)
  Signal 14: 3 (delay: 3.199721)
  Signal 15: 5 (delay: 3.869463)
  Signal 16: 8 (delay: 1.240504)
  Signal 17: 2 (delay: 4.661430)
  Signal 18: 3 (delay: 2.628724)
  Signal 19: 5 (delay: 3.836867)
  Signal 20: 7 (delay: 4.943584)
Signal ended
Blocked queue
